## Ejemplo Introductorio

In [0]:
generation = "mod4gen13"

In [0]:
data = spark.read.format("libsvm").load(f"/mnt/{generation}/input/sample_linear_regression_data.txt", inferSchema=True, header=True)

In [0]:
data.count(), data.columns

Es común en la práctica tener datos de entrenamiento poco densos. MLlib admite la lectura de registros de entrenamiento almacenados en formato `LIBSVM`, que es un formato de texto predeterminado en el que cada línea representa un vector de características dispersas etiquetadas.

In [0]:
data.display()

In [0]:
data.show()

In [0]:
type(data)

In [0]:
# Train/Test split
train, test = data.randomSplit([0.8, 0.2])

In [0]:
train.count(), test.count()

In [0]:
train.show()

In [0]:
test.show()

In [0]:
unlabeled_test = test.select("features")

In [0]:
unlabeled_test.show()

El formato que espera Spark MLlib son dos columnas, una con la variable objetivo y otra con el listado de variables.

La columna asociada a la variable objetivo debe ser de tipo numérico, ya sea un valor continuo para regresión o un valor discreto que coincida con una categoría de clasificación. Más adelante hablaremos de algoritmos de aprendizaje no supervisado que por su naturaleza no utilizan ni requieren etiquetas.

La columna de características tiene dentro un vector con todas las variables que pertenecen a esa fila. Por lo general, lo que se suele hacer es combinar las diversas variables con las que contamos en una sola columna de características.

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
help(LinearRegression)

In [0]:
# Estos son los valores predeterminados para 'featuresCol', 'labelCol' y 'predictionCol'
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [0]:
# Entrenamiento de modelo
lr_model = lr.fit(train)

In [0]:
# Imprime los coeficientes y el intercepto para la regresión lineal
print("Coefficientes: {}".format(str(lr_model.coefficients)))
print("Intercepto:{}".format(str(lr_model.intercept)))

In [0]:
# Resumen del modelo
train_results = lr_model.summary

In [0]:
type(train_results)

In [0]:
train_results.residuals.display()

In [0]:
# Predicciones
predictions = lr_model.transform(unlabeled_test)

In [0]:
predictions.show()

In [0]:
test_results = lr_model.evaluate(test)

In [0]:
# Evaluación train
print("RMSE: {}".format(train_results.rootMeanSquaredError))
print("r2: {}".format(train_results.r2))

In [0]:
# Evaluación test
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("r2: {}".format(test_results.r2))